In [1]:
import os
import awswrangler as wr
import pandas as pd
import numpy as np

In [2]:
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [11]:
# AWS directories
s3_root = 's3://crc-data-lake-research/sma/jimmy_woo'
research_database = 'crc_data_lake_research'
results_path = '{}/temp_resuts'.format(s3_root)
table_path = '{}/market_share/tier_4_level/'.format(s3_root)
table_name = 'sma_market_share_tier_4'

In [5]:
query = """
SELECT DISTINCT tier_4
FROM crc_data_lake_structured.ised_grid
ORDER BY 1
"""

tiers = wr.athena.read_sql_query(
    query,
    database=database,
    s3_output=results_path,
    keep_files=False,
)

tiers = list(tiers['tier_4'])

In [6]:
def tier_4_query(tier):
    query_parameters = {'tier': tier}

    query = """
    SELECT
      ised_grid.tier_2,
      ised_grid.tier_3,
      ised_grid.tier_4,
      tutela.connection_serviceproviderbrandname as service_provider,
      band_plan.band_short AS band,
      band_plan.block,
      COUNT(*) as num_data_pts
    FROM crc_data_lake_structured.tutela
    JOIN crc_data_lake_structured.ised_grid
      ON ST_Within(
              ST_Point(tutela.location_longitude, tutela.location_latitude),
              ST_GeometryFromText(ised_grid.geometry)
      )
    JOIN crc_data_lake_structured.band_plan
      ON (
        tutela.connection_mobilefrequency >= band_plan.lower_frequency_start_mhz
        AND tutela.connection_mobilefrequency < band_plan.lower_frequency_stop_mhz
      )
      OR (
        tutela.connection_mobilefrequency >= band_plan.upper_frequency_start_mhz
        AND tutela.connection_mobilefrequency < band_plan.upper_frequency_stop_mhz
      )
    JOIN crc_data_lake_structured.spectrum_area_license
      ON spectrum_area_license.area IN (
          ised_grid.tier_2,
          ised_grid.tier_3, 
          ised_grid.tier_4
        )
      AND spectrum_area_license.band = band_plan.band_short
      AND spectrum_area_license.block = band_plan.block
    WHERE
      ised_grid.tier_4 = ':tier;'
      AND tutela.country = 'Canada'
      AND tutela.connection_technology IN ('LTE', 'LTE_CA')
    GROUP BY 1, 2, 3, 4, 5, 6
    ORDER BY 7 DESC
    """

    df = wr.athena.read_sql_query(
            query,
            database=research_database,
            s3_output=results_path,
            keep_files=False,
            params=query_parameters
        )

    df = df.dropna(subset=['service_provider'])
    
    return df

In [7]:
def compute_market_share(df):
    total = np.sum(df['num_data_pts'])
    df['market_share'] = df['num_data_pts'] / total
        
    return df

In [12]:
err = {}

for tier in tiers:
    try:
        df = tier_4_query(tier)
        df = compute_market_share(df)
        
        wr.s3.to_parquet(
            df,
            path=table_path,
            dataset=True,
            database=research_database,
            table=table_name
        )
    except Exception as e:
        err[tier] = e

if err:
    err_cols = ['tier_4', 'err_msg']
    err_df = pd.DataFrame.from_dict(data=err, orient='index').reset_index()
    err_df.columns = err_cols

    wr.s3.to_json(
        err_df,
        path='{}/market_share/tier_4_err.json'.format(s3_root),
        dataset=False
    )